In [7]:
import json
import pandas as pd
import warnings
import seaborn as sns
from pyprojroot import here

methods = ["TimeGAN", "Time-Transformer", "TransFusion", "TTS-GAN"]
datasets = ["D2", "D3", "D4", "D5", "D6", "D7"]

rows = []

for method in methods:
    for dataset in datasets:
        file_candidates = list(here('result').glob(f'numeric_{method}_{dataset}_*.json'))
        if len(file_candidates) != 1:
            warnings.warn(f"Ignoring {method} {dataset}: Expected one result file for {method} {dataset}, instead matched {file_candidates}.")
            continue
        file_path = file_candidates[0]
        
        with open(file_path) as f:
            data = json.load(f)
        
        rows.append(((method, dataset), data))

df = pd.DataFrame.from_dict(dict(rows), orient="index")
df.index = pd.MultiIndex.from_tuples(df.index, names=["Method", "Dataset"])

df

DS        PS         C-FID       MDD  \
Method           Dataset                                               
TimeGAN          D2       0.112879  0.038129  1.905527e-02  0.434670   
                 D3       0.362715  0.041257  5.028699e-02  0.870274   
                 D4       0.116530  0.133363  1.051006e-01  0.766039   
                 D5       0.472527  0.285059  1.545321e-01  0.382266   
                 D6       0.500000  0.309733  6.350298e-01  0.741083   
                 D7       0.011219  0.000890  8.908389e-04  2.217216   
Time-Transformer D2       0.315152  0.085012  2.582186e-01  0.627615   
                 D3       0.312012  0.072252  1.776648e-01  0.747522   
                 D4       0.124348  0.066113  4.560189e-02  0.517450   
                 D5       0.499859  0.326896  1.473801e-01  0.389958   
                 D6       0.500000  0.251982  4.563424e-01  0.614462   
                 D7       0.480366  0.002254  1.516585e-03  9.074205   
TransFusion      D2       0.150000  0.037576  9.027198e-02  0.793641   
                 D3       0.000780  0.037371  4.399024e-03  0.404956   
                 D4       0.059568  0.049407  1.190948e-02  0.291926   
                 D5       0.118625  0.251417  9.846010e-03  0.227218   
                 D6       0.144476  0.248861  4.271202e-03  0.198377   
                 D7       0.202693  0.000563  1.954121e-04  7.928880   
TTS-GAN          D2       0.000000  0.051607  1.431463e+17  1.003476   
                 D3       0.498440  0.378110  3.351009e+11  1.020235   
                 D4       0.495905  0.374939  1.001744e+14  1.013873   
                 D5       0.500000  0.445550  9.089393e+10  1.010267   
                 D6       0.500000  0.401835  1.410485e+12  1.014350   
                 D7       0.500000  0.629734  2.583964e+11  7.517625   

                               ACD         SD            KD            ED  \
Method           Dataset                                                    
TimeGAN          D2       0.171891   0.482856      2.351818  1.189109e+00   
                 D3       0.143190   0.262877      2.341035  2.646002e+00   
                 D4       0.127232   0.194281      0.496604  2.857787e+00   
                 D5       0.451997   0.263064      1.007865  1.057500e+00   
                 D6       2.335000   0.547243      0.918373  2.610278e+00   
                 D7       1.864680  73.477606   8942.861949  5.024082e-02   
Time-Transformer D2       0.247822   0.592418      2.517755  1.307718e+00   
                 D3       0.486708   0.522732      2.953833  2.561868e+00   
                 D4       0.308017   0.261743      0.376560  2.692303e+00   
                 D5       0.266162   0.204340      0.836956  9.502726e-01   
                 D6       0.701291   0.417475      1.213679  2.451943e+00   
                 D7       1.281966  73.553105   8942.827292  1.069579e-01   
TransFusion      D2       0.033649   0.533438      2.629961  1.072507e+00   
                 D3       0.111957   0.175680      0.824398  2.779363e+00   
                 D4       0.038166   0.118325      0.242787  2.833876e+00   
                 D5       0.062980   0.108366      0.426964  1.012462e+00   
                 D6       0.106999   0.035632      0.134695  2.546032e+00   
                 D7       0.182951  60.550151  20819.667549  4.560368e-02   
TTS-GAN          D2       4.518805   1.477441      3.153213  1.590050e+08   
                 D3       7.754373   1.258643      3.018282  1.057488e+06   
                 D4       7.850430   0.330419      1.880705  1.455607e+07   
                 D5       4.522544   1.396577      1.732293  8.644544e+04   
                 D6       6.084712   0.826237      1.642648  1.417871e+06   
                 D7       3.069357  73.693896   8943.578376  5.247503e+05   

                                   DTW  
Method           Dataset                
TimeGAN          D2       3.068079e+00  
                 D3       6.67951

In [8]:

cm = sns.diverging_palette(h_neg=130, h_pos=0, as_cmap=True)
#cm = sns.light_palette("green", as_cmap=True, reverse=True)

df.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"])\
    .style\
        .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
        .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
        .background_gradient(cmap=cm, vmin=0, vmax=10, subset=["DTW"])

In [9]:

df.style\
    .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
    .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
    .background_gradient(cmap=cm, vmin=0, vmax=15, subset=["DTW"])

In [19]:
df_no_tts = df[df.index.get_level_values("Method") != "TTS-GAN"]
df_no_tts \
    .style\
        .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
        .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
        .background_gradient(cmap=cm, vmin=0, vmax=15, subset=["DTW"])

In [20]:
df_no_tts.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"])\
    .style\
        .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
        .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
        .background_gradient(cmap=cm, vmin=0, vmax=10, subset=["DTW"])

In [10]:
# Min-Max scale by metric and reverse score so 1 is best and 0 is worst
df_minmax = 1 - ((df - df.min()) / (df.max() - df.min()))


df_mean_by_metric = df_minmax.groupby('Method').mean()


df_spider_by_metric = df_mean_by_metric.reset_index().melt(id_vars="Method", var_name="Metric")
df_spider_by_metric = df_spider_by_metric[df_spider_by_metric["Method"] != "TTS-GAN"] # remove tts-gan

import plotly.express as px

px.line_polar(df_spider_by_metric, r="value", theta="Metric", color="Method", line_close=True)

In [15]:
df_mean_by_dataset = df_minmax.stack().unstack(level=1).rename_axis(("Method","Metric")).groupby("Method").mean()

df_spider_by_dataset = df_mean_by_dataset.reset_index().melt(id_vars="Method")
df_spider_by_dataset = df_spider_by_dataset[df_spider_by_dataset["Method"] != "TTS-GAN"] # remove tts-gan



px.line_polar(df_spider_by_dataset, r="value", theta="Dataset", color="Method", line_close=True)

In [12]:
rankings = df.unstack(level=1).rank()

In [13]:
ranking_by_metric = rankings.stack().reset_index().groupby("Method").mean()
spider_by_metric = ranking_by_metric.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_metric, r="value", theta="variable", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [4.1, 1], "dtick": 1}})

In [14]:
ranking_by_dataset = rankings.stack(level=0).reset_index().groupby("Method").mean()
spider_by_dataset = ranking_by_dataset.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_dataset, r="value", theta="Dataset", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [4.1,1], "dtick": 1}})